In [95]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns

In [96]:
background_path = 'Background_clean.csv'
log_path = 'Logs_clean.csv'

background_df = pd.read_csv(background_path)
log_df = pd.read_csv(log_path)

In [97]:
background_df.head()

,index,user_id,gender,age_group,LSM_group1,share_your_home_with,including_yourself_how_many_people_do_you_share_your_home_with,do_to_try_to_stay_healthy,drives_you_to_have_a_healthy_lifestyle,eat_or_drink,steps_to_eat_drink_more_healthily,how_regularly_do_you_exercise,exercise_do_you_enjoy,do_to_help_you_sleep,before_you_go_to_sleep
0,1,3.892140e+08,Female,25-34,High,"[""Children over 18 years old""]",4.0,"[""Stay hydrated""]","[""To make the most out of/enjoy life""]","[""None of the above""]","[""Eating out at places that have healthy optio...",3+ times a week,"[""Running"", ""Yoga/Pilates""]","[""Have a bath"", ""Have a malted milk drink""]","[""Browsing on my cell phone / tablet"", ""Caffei..."
1,3,0.000000e+00,Female,45+,High,"[""Partner / spouse""]",1,"[""Drink lots of water / fluids"", ""Eat healthil...","[""To manage a health / medical condition"", ""Pr...","[""Flexitarian: eat meat but control the amount...","[""Intermittent fasting""]",Never,NaN,"[""Have herbal tea"", ""Have a bath""]","[""Caffeine"", ""Heavy meal"", ""Other""]"
2,4,2.647339e+09,Male,25-34,High,"[""Parents"", ""Other family members""]",4.0,"[""Taking time to relax"", ""Eat healthily"", ""Dri...","[""To make the most of treats / indulgent momen...","[""Low sugar"", ""Low salt"", ""Clean eating: eat m...","[""Trying to have a variety of colors on my pla...",3+ times a week,"[""Walking"", ""Cycling"", ""Hiking""]","[""Have warm milk"", ""Other""]","[""Heavy meal"", ""Caffeine""]"
3,5,9.512227e+08,Female,45+,High,"[""Partner / spouse""]",3.0,"[""Eat healthily"", ""Exercise regularly"", ""Make ...","[""To improve my sleep"", ""To manage a health / ...","[""Low salt""]","[""Cooking more meals at home""]",Twice a week,"[""Hiking""]","[""No, none of these""]","[""Caffeine""]"
4,6,1.056650e+09,Male,35-44,Medium,"[""Children under 18 years old"", ""Partner / spo...",5+,"[""Make sure I get enough sleep"", ""Taking time ...","[""To make the most out of/enjoy life"", ""To imp...","[""None of the above""]","[""Cooking more meals at home""]",Never,NaN,"[""No, none of these""]","[""Caffeine""]"


In [98]:
log_df.head()

,index,submission_id,user_id,location_name,submission_timestamp,where_are_you_eating_at_the_moment,who_are_you_with,where_are_you_eating_at_the_moment.1,which_of_the_following_best_describes_where_you_got_your_food_from_today,what_else_if_anything_are_you_doing_while_eating_your_food_for_example_are_you_also_watching_tv_checking_social_media_cooking_etc_coded,who_prepared_the_food,level2dish_coded,what_are_the_main_reasons_that_you_chose_this_food_right_now_if_you_are_having_more_than_one_item_please_write_about_the_main_item_you_are_having_please_be_as_detailed_as_you_can_coded,what_other_food_if_any_did_you_consider_having_instead_coded,finally_which_best_describes_the_food_you_are_having_today
0,0,1,3.892140e+08,South Africa,2019-01-31 15:11:16.447589,At school / work,"[""Alone""]",At school / work,I had it at home / brought it from home,"[""Working / studying""]",I prepared it myself from scratch,"[""Pork""]","[""I like the taste""]","[""Nothing""]",Lunch
1,1,4,2.647339e+09,South Africa,2019-01-24 18:30:12.830288,At my home,"[""Alone""]",At my home,I had it at home / brought it from home,"[""Watching TV"", ""Browsing social media""]",Someone else prepared it,"[""Other potatoes/ sweet potatoes"", ""Bread/ rol...","[""It's a healthy choice"", ""Cheap""]","[""Meat""]",Dinner
2,2,6,1.056650e+09,South Africa,2019-08-22 15:13:50.876941,On the go / outdoors,"[""My spouse/partner"", ""My children""]",On the go / outdoors,Supermarket / Hypermarket / Grocery Store,"[""Spending time with family""]",It was premade / ready to eat,"[""Biscuits""]","[""Quick"", ""Kids/ family like it""]","[""Nothing""]",Snack
3,3,7,2.771745e+09,South Africa,2019-05-24 16:21:32.007399,At my home,"[""Alone""]",At my home,NaN,"[""Talking to someone"", ""Spending time with fri...",Someone else prepared it,"[""Cake""]","[""I like the taste""]","[""Nothing""]",Snack
4,4,8,3.427770e+09,South Africa,2019-02-13 04:02:58.491098,At my home,"[""Alone""]",At my home,I had it at home / brought it from home,"[""Watching TV""]",I prepared it myself from scratch,"[""Bread/ roll/ baguette"", ""Instant noodles""]","[""I like the taste""]","[""Nothing""]",Snack


In [99]:
def get_index_in_set(set_val, item):
    index = 0
    return_index = -1
    for set_item in set_val:
        if set_item == item:
            return_index = index
        index += 1
    return return_index

In [100]:
def list_to_multi_hot(pd_series_list):
    unique_values = set()
    for _, value_list in pd_series_list.iteritems():
        if pd.notnull(value_list):
            for value in ast.literal_eval(value_list):
                unique_values.add(value)
    unique_count = len(unique_values)
    multi_hot = []
    for _, value_list in pd_series_list.iteritems():
        multi_hot_item = [0] * unique_count
        if pd.notnull(value_list):
            for value in ast.literal_eval(value_list):
                index = get_index_in_set(unique_values, value)
                if index != -1:
                    multi_hot_item[index] = 1
        multi_hot.append(multi_hot_item)
    multi_hot = np.array(multi_hot)
    multi_hot = pd.DataFrame(multi_hot)
    multi_hot.columns = [pd_series_list.name + '_' + choice for choice in list(unique_values)]
    return multi_hot

In [123]:
background_df.iloc[325:330, :].exercise_do_you_enjoy

325       Football
326            NaN
327            NaN
328    ["Running"]
329            NaN
Name: exercise_do_you_enjoy, dtype: object

In [122]:
'eat_or_drink'
'exercise_do_you_enjoy'
list_to_multi_hot(background_df.iloc[325:330, :].exercise_do_you_enjoy)

ValueError: malformed node or string: <ast.Name object at 0x7fb41e408580>

In [49]:
list_columns = ['share_your_home_with', 'do_to_try_to_stay_healthy',
               'drives_you_to_have_a_healthy_lifestyle', 'eat_or_drink',
               'steps_to_eat_drink_more_healthily', 'exercise_do_you_enjoy',
               'do_to_help_you_sleep', 'before_you_go_to_sleep']

for col_name in list_columns:
    multi_hot = list_to_multi_hot(background_df[col_name])
    del background_df[col_name]
    background_df = background_df.join(multi_hot)

SyntaxError: invalid syntax (<unknown>, line 1)

In [50]:
background_df.head()

,index,user_id,gender,age_group,LSM_group1,including_yourself_how_many_people_do_you_share_your_home_with,eat_or_drink,steps_to_eat_drink_more_healthily,how_regularly_do_you_exercise,exercise_do_you_enjoy,...,"drives_you_to_have_a_healthy_lifestyle_Specific event (e.g. holiday, wedding)",drives_you_to_have_a_healthy_lifestyle_To improve my sleep,drives_you_to_have_a_healthy_lifestyle_To support my family,drives_you_to_have_a_healthy_lifestyle_To make the most out of/enjoy life,drives_you_to_have_a_healthy_lifestyle_Other,drives_you_to_have_a_healthy_lifestyle_Nothing,drives_you_to_have_a_healthy_lifestyle_To live longer,drives_you_to_have_a_healthy_lifestyle_Feeling good about myself,drives_you_to_have_a_healthy_lifestyle_Have more energy,drives_you_to_have_a_healthy_lifestyle_Lack of money
0,1,3.892140e+08,Female,25-34,High,4.0,"[""None of the above""]","[""Eating out at places that have healthy optio...",3+ times a week,"[""Running"", ""Yoga/Pilates""]",...,0,0,0,1,0,0,0,0,0,0
1,3,0.000000e+00,Female,45+,High,1,"[""Flexitarian: eat meat but control the amount...","[""Intermittent fasting""]",Never,NaN,...,0,0,0,0,0,0,0,0,0,0
2,4,2.647339e+09,Male,25-34,High,4.0,"[""Low sugar"", ""Low salt"", ""Clean eating: eat m...","[""Trying to have a variety of colors on my pla...",3+ times a week,"[""Walking"", ""Cycling"", ""Hiking""]",...,0,1,0,0,0,0,0,0,0,0
3,5,9.512227e+08,Female,45+,High,3.0,"[""Low salt""]","[""Cooking more meals at home""]",Twice a week,"[""Hiking""]",...,0,1,0,0,0,0,1,0,0,0
4,6,1.056650e+09,Male,35-44,Medium,5+,"[""None of the above""]","[""Cooking more meals at home""]",Never,NaN,...,0,1,0,1,0,0,1,0,0,0


,0
0,"[""Children over 18 years old""]"
1,"[""Partner / spouse""]"
2,"[""Parents"", ""Other family members""]"
3,"[""Partner / spouse""]"
4,"[""Children under 18 years old"", ""Partner / spo..."
...,...
1011,"[""Partner / spouse"", ""Children under 18 years ..."
1012,"[""Partner / spouse""]"
1013,"[""I live alone""]"
1014,"[""Partner / spouse"", ""Children under 18 years ..."


In [12]:
log_df.head()

,index,submission_id,user_id,location_name,submission_timestamp,where_are_you_eating_at_the_moment,who_are_you_with,where_are_you_eating_at_the_moment.1,which_of_the_following_best_describes_where_you_got_your_food_from_today,what_else_if_anything_are_you_doing_while_eating_your_food_for_example_are_you_also_watching_tv_checking_social_media_cooking_etc_coded,who_prepared_the_food,level2dish_coded,what_are_the_main_reasons_that_you_chose_this_food_right_now_if_you_are_having_more_than_one_item_please_write_about_the_main_item_you_are_having_please_be_as_detailed_as_you_can_coded,what_other_food_if_any_did_you_consider_having_instead_coded,finally_which_best_describes_the_food_you_are_having_today
0,0,1,3.892140e+08,South Africa,2019-01-31 15:11:16.447589,At school / work,"[""Alone""]",At school / work,I had it at home / brought it from home,"[""Working / studying""]",I prepared it myself from scratch,"[""Pork""]","[""I like the taste""]","[""Nothing""]",Lunch
1,1,4,2.647339e+09,South Africa,2019-01-24 18:30:12.830288,At my home,"[""Alone""]",At my home,I had it at home / brought it from home,"[""Watching TV"", ""Browsing social media""]",Someone else prepared it,"[""Other potatoes/ sweet potatoes"", ""Bread/ rol...","[""It's a healthy choice"", ""Cheap""]","[""Meat""]",Dinner
2,2,6,1.056650e+09,South Africa,2019-08-22 15:13:50.876941,On the go / outdoors,"[""My spouse/partner"", ""My children""]",On the go / outdoors,Supermarket / Hypermarket / Grocery Store,"[""Spending time with family""]",It was premade / ready to eat,"[""Biscuits""]","[""Quick"", ""Kids/ family like it""]","[""Nothing""]",Snack
3,3,7,2.771745e+09,South Africa,2019-05-24 16:21:32.007399,At my home,"[""Alone""]",At my home,NaN,"[""Talking to someone"", ""Spending time with fri...",Someone else prepared it,"[""Cake""]","[""I like the taste""]","[""Nothing""]",Snack
4,4,8,3.427770e+09,South Africa,2019-02-13 04:02:58.491098,At my home,"[""Alone""]",At my home,I had it at home / brought it from home,"[""Watching TV""]",I prepared it myself from scratch,"[""Bread/ roll/ baguette"", ""Instant noodles""]","[""I like the taste""]","[""Nothing""]",Snack
